# Group Equivariant Graph Tutorial

## Introduction

In this tutorial, we present the different functionnalities of our implementation of the group equivariant graph. Such a graph is a k-NN graph representing the structure of a group, a set equipped with a binary operation called the group product. Each vertex of the graph represents an element of the group and the edge's weights corresponds to the Riemannian distance between these element. 

In [ ]:
import torch
import numpy as np
import math

## Package

The `gechebnet` package implements such a graph and have the basic functions to interact with them. For the most part, it uses PyTorch for tensor operations and PyKeops to achieve a memory and time efficient graph construction.

## Graph class

The main class of the package is the `Graph` class. It stores a graph in a very efficient way using `node_index`, `edge_index` and `edge_weight` attributes. Very easily, we can get important properties of a graph with `num_nodes` or `num_edges`. Other method, properties and attributes are available, we will see them later. However, this class just gives the main structure of a graph, the group equivariant graph inherits from this class.

An important remark is that our graphs are necessarily symmetric, that's why we refer the `knn` parameter as the **maximum** number of neighbours of a vertex.

## SE(2) Group Equivariant Graph

The SE(2) group equivariant graph is the oldest group equivariant graph we have implemented. To create such a graph, different variables have to be speficied:
- `grid_size`: the limit of the space in 2 dimensional euclidean space. It corresponds to the spatial domain.
- `nx3`: the resolution of the discretization of the orientation axis. It corresponds to the orientation domain.
- `knn`: the maximum number of neighbors of each vertex. 
- `sigmas`: the anisotropy's parameters for the computation of the Riemannian distance between vertices.
- `weight_kernel`: the weight kernel, that is a function taking as input the squared Riemannian distance and returning a weight.
- `kappa`: the edge's compression rate, the rate of edges to drop during random compression.

In [ ]:
from gechebnet.graph.graph import SE2GEGraph

In [ ]:
xi = 0.001
eps = .58

graph = SE2GEGraph(
    grid_size=(28, 28),
    nx3=9,
    knn=128,
    sigmas=(xi / eps, xi, 1.0),
    weight_kernel=lambda sqdistc, sigmac: torch.exp(-sqdistc / sigmac),           # gaussian
    #weight_kernel = lambda sqdistc, sigmac: torch.exp(-torch.sqrt(sqdistc/sigmac)), # laplace
    #weight_kernel = lambda sqdistc, sigmac: 1 / (1 + sqdistc / sigmac),        # cauchy
    kappa=0.,
)

In [ ]:
f"The SE(2) graph has {graph.num_nodes} vertices and {graph.num_edges} edges"

## Visualization

To visualize the vertices of a graph or the neighborhood of a vertex, you can use the function `visualize_graph`, `visualize_neighborhood`. The former can also be used with an optional `signal` parameter to plot a signal on a graph.

In [ ]:
from gechebnet.graph.plot import visualize_graph, visualize_neighborhood

In [ ]:
_ = visualize_graph(graph)

In [ ]:
_ = visualize_neighborhood(graph, graph.centroid_index)

## Fourier basis

A discrete graph can be seen as an approximation of a continuous manifold. To evaluate the quality of this approximation, we can rely on the eigen space of the graph (an other work to speak about its spectrum). The eigen decomposition of the graph should give the same eigen space as the continuous manifold that we are trying to approximate. It is a well-known fact that the eigen functions of the Euclidean space are the fourier basis (cosine and sine functions). In the case of the SO(3) group, the eigen functions are the so-called harmonic spherics.

In [ ]:
eigenvec = torch.from_numpy(graph.eigen_space[1])
_ = visualize_graph(graph, eigenvec[:, 3])

## Diffusion process

A tool to visualize a diffusion process on a graph is also available. The computation of this diffusion relies on the graph spectral theory and the eigen decomposition of the graph laplacian. The `visualize_diffusion_process` takes as parameters the time constants of the diffusion `times`, the diffusion kernel `diff_kernel` ($exp (-\tau \lambda)$ for the heat diffusion) and the initial signal on the graph that we want to diffuse `f0` (a centroid node centered dirac function by default). The diffusion process is save as a gif image.

In [ ]:
from gechebnet.graph.plot import visualize_diffusion_process

In [ ]:
times = np.arange(0., 200., 20.)
diff_kernel = lambda x, t: np.exp(-t * x)
visualize_diffusion_process(graph, f0=graph.dirac(0), times=times, diff_kernel=diff_kernel, file_name= "heat_diffusion.gif")

In [ ]:
times = np.arange(0., 200., 20.)
diff_kernel = lambda x, t: (x/2.) ** t # rescale eigenvalues to avoid explosion
visualize_diffusion_process(graph, f0=graph.dirac(0), times=times, diff_kernel=diff_kernel, file_name= "power_diffusion.gif")

In [ ]:
w = torch.rand(1000)
kappa=1

torch.multinomial(1-w, 50)

In [ ]:
torch.bernoulli((1-kappa)*w).bool()